In [6]:
import os
import cv2
import numpy as np

def compute_rgb_mhi(video_path, duration=1.0, threshold=64):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    mhi_duration = int(fps * duration)

    timestamp = 0
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return None

    red_mhi = np.zeros_like(frame, dtype=np.float32)
    green_mhi = np.zeros_like(frame, dtype=np.float32)
    blue_mhi = np.zeros_like(frame, dtype=np.float32)


    prev_frame = frame

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_frame = frame
        frame_diff = cv2.absdiff(current_frame, prev_frame)
        _, motion_mask = cv2.threshold(frame_diff, threshold, 1, cv2.THRESH_BINARY)
        timestamp += 1

        # Update MHIs based on color segments
        frame_index = timestamp % mhi_duration
        if frame_index < 21:
            update_mhi(motion_mask, timestamp, mhi_duration, red_mhi)
        elif frame_index < 43:
            update_mhi(motion_mask, timestamp, mhi_duration, green_mhi)
        else:
            update_mhi(motion_mask, timestamp, mhi_duration, blue_mhi)

        prev_frame = current_frame

    cap.release()

    # Combine color MHIs into RGBMHI
    rgb_mhi = cv2.merge([blue_mhi, green_mhi, red_mhi])  # Combine in BGR order
    return cv2.normalize(rgb_mhi, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

def update_mhi(motion_mask, timestamp, mhi_duration, mhi):
    if mhi is None:
        mhi = np.zeros_like(motion_mask, dtype=np.float32)
    mhi = np.maximum(mhi - 1 / mhi_duration, 0)
    mhi[motion_mask == 1] = timestamp

# Rest of your code remains the same, calling compute_rgb_mhi instead of compute_mhi


In [7]:

# Path to the folder containing videos
video_folder = r'/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor'

# List all files in the folder
video_files = os.listdir(video_folder)

# Iterate over video files
for video_file in video_files:
    # Check if the file is an MKV video
    if video_file.endswith('.mkv'):
        video_path = os.path.join(video_folder, video_file)
        # Compute the MHI for the provided video
        mhi_image = compute_rgb_mhi(video_path)
        
        # Check if MHI was successfully computed
        if mhi_image is not None:
            # Create a folder named 'mhi' if it doesn't exist
            mhi_folder = os.path.join(video_folder, 'rgb-mhi')
            if not os.path.exists(mhi_folder):
                os.makedirs(mhi_folder)
            
            # Save the resulting MHI image to the 'mhi' folder
            mhi_filename = f'mhi_{os.path.splitext(video_file)[0]}.png'
            mhi_path = os.path.join(mhi_folder, mhi_filename)
            cv2.imwrite(mhi_path, mhi_image)
            print(f"MHI saved: {mhi_path}")
        else:
            print("Failed to compute MHI for", video_file)

error: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:703: error: (-215:Assertion failed) image.channels() == 1 || image.channels() == 3 || image.channels() == 4 in function 'imwrite_'


In [10]:
import os
import cv2
import numpy as np

def compute_rgb_mhi(video_path, duration=1.0, threshold=64):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    mhi_duration = int(fps * duration)

    timestamp = 0
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return None

    red_mhi = np.zeros_like(frame, dtype=np.float32)
    green_mhi = np.zeros_like(frame, dtype=np.float32)
    blue_mhi = np.zeros_like(frame, dtype=np.float32)

    prev_frame = frame

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_frame = frame
        frame_diff = cv2.absdiff(current_frame, prev_frame)
        _, motion_mask = cv2.threshold(frame_diff, threshold, 1, cv2.THRESH_BINARY)
        timestamp += 1

        # Update MHIs based on color segments
        frame_index = timestamp % mhi_duration
        if frame_index < 21:
            update_mhi(motion_mask, timestamp, mhi_duration, red_mhi)
        elif frame_index < 43:
            update_mhi(motion_mask, timestamp, mhi_duration, green_mhi)
        else:
            update_mhi(motion_mask, timestamp, mhi_duration, blue_mhi)

        prev_frame = current_frame

    cap.release()

    # Combine color MHIs into RGBMHI
    rgb_mhi = cv2.merge([blue_mhi, green_mhi, red_mhi])  # Combine in BGR order
    normalized_rgb_mhi = cv2.normalize(rgb_mhi, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    # Check if the image is grayscale and convert to RGB if necessary
    if len(normalized_rgb_mhi.shape) == 2:
        normalized_rgb_mhi = cv2.cvtColor(normalized_rgb_mhi, cv2.COLOR_GRAY2RGB)

    return normalized_rgb_mhi

def update_mhi(motion_mask, timestamp, mhi_duration, mhi):
    if mhi is None:
        mhi = np.zeros_like(motion_mask, dtype=np.float32)
    mhi = np.maximum(mhi - 1 / mhi_duration, 0)
    mhi[motion_mask == 1] = timestamp

# Path to the folder containing videos
video_folder = r'/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor'

# List all files in the folder
video_files = os.listdir(video_folder)

# Iterate over video files
for video_file in video_files:
    # Check if the file is an MKV video
    if video_file.endswith('.mkv'):
        video_path = os.path.join(video_folder, video_file)
        # Compute the MHI for the provided video
        mhi_image = compute_rgb_mhi(video_path)
        
        # Check if MHI was successfully computed
        if mhi_image is not None:
            # Create a folder named 'mhi' if it doesn't exist
            mhi_folder = os.path.join(video_folder, 'rgb-mhi')
            if not os.path.exists(mhi_folder):
                os.makedirs(mhi_folder)
            
            # Save the resulting MHI image to the 'mhi' folder
            mhi_filename = f'mhi_{os.path.splitext(video_file)[0]}.png'
            mhi_path = os.path.join(mhi_folder, mhi_filename)
            cv2.imwrite(mhi_path, mhi_image)
            print(f"MHI saved: {mhi_path}")
        else:
            print("Failed to compute MHI for", video_file)


error: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:703: error: (-215:Assertion failed) image.channels() == 1 || image.channels() == 3 || image.channels() == 4 in function 'imwrite_'


In [33]:
import os
import cv2
import numpy as np

def compute_rgb_mhi(video_path,frame=64, duration=1.0, threshold=64):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    mhi_duration = int(fps * duration)

    timestamp = 0
    red_mhi = np.zeros_like(frame, dtype=np.float32)
    green_mhi = np.zeros_like(frame, dtype=np.float32)
    blue_mhi = np.zeros_like(frame, dtype=np.float32)

    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return None

    prev_frame = frame

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_frame = frame
        frame_diff = cv2.absdiff(current_frame, prev_frame)
        _, motion_mask = cv2.threshold(frame_diff, threshold, 1, cv2.THRESH_BINARY)
        timestamp += 1

        # Update MHIs based on color segments
        frame_index = timestamp % mhi_duration
        if frame_index < 21:
            update_mhi(motion_mask, timestamp, mhi_duration, red_mhi)
        elif frame_index < 43:
            update_mhi(motion_mask, timestamp, mhi_duration, green_mhi)
        else:
            update_mhi(motion_mask, timestamp, mhi_duration, blue_mhi)

        prev_frame = current_frame

    cap.release()

    # Combine color MHIs into RGBMHI
    rgb_mhi = cv2.merge([blue_mhi, green_mhi, red_mhi])  # Combine in BGR order
    normalized_rgb_mhi = cv2.normalize(rgb_mhi, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    # Check if the image is grayscale and convert to RGB if necessary
    if len(normalized_rgb_mhi.shape) == 2:
        normalized_rgb_mhi = cv2.cvtColor(normalized_rgb_mhi, cv2.COLOR_GRAY2RGB)

    return normalized_rgb_mhi

def update_mhi(motion_mask, timestamp, mhi_duration, mhi):
    if mhi is None:
        mhi = np.zeros_like(motion_mask, dtype=np.float32)
    mhi = np.maximum(mhi - 1 / mhi_duration, 0)
    mhi[motion_mask == 1] = timestamp

# Path to the folder containing videos
video_folder = r'/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor'

# List all files in the folder
video_files = os.listdir(video_folder)

# Iterate over video files
for video_file in video_files:
    # Check if the file is an MKV video
    if video_file.endswith('.mkv'):
        video_path = os.path.join(video_folder, video_file)
        # Compute the MHI for the provided video
        mhi_image = compute_rgb_mhi(video_path)

        # Check if MHI was successfully computed
        if mhi_image is not None:
            # Create a folder named 'mhi' if it doesn't exist
            mhi_folder = os.path.join(video_folder, 'rgb-mhi')
            if not os.path.exists(mhi_folder):
                os.makedirs(mhi_folder)

            mhi_filename = f'mhi_{os.path.splitext(video_file)[0]}.png'
            mhi_path = os.path.join(mhi_folder, mhi_filename)
            cv2.imwrite(mhi_path, mhi_image)
            print(f"MHI saved: {mhi_path}")
        else:
            print("Failed to compute MHI for", video_file)


TypeError: 'numpy.float64' object does not support item assignment

In [31]:
# Define a function to compute Motion History Images (MHIs)
def compute_mhi(video_path, duration=1.0, threshold=64):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Calculate the max value for MHI based on duration and FPS of the video
    mhi_duration = int(fps * duration)  # Duration in frames
    timestamp = 0  # Current time stamp
    mhi = None  # Motion History Image
    
    # Read the first frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return None
    # Convert to grayscale
    prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Loop through the video
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Compute the absolute difference between the current frame and the previous frame
        frame_diff = cv2.absdiff(current_frame, prev_frame)
        # Threshold the difference
        _, motion_mask = cv2.threshold(frame_diff, threshold, 1, cv2.THRESH_BINARY)
        timestamp += 1  # Update the timestamp
        
        # Update MHI
        if mhi is None:
            mhi = np.zeros_like(prev_frame, dtype=np.float32)
        # Decrease the MHI value for all pixels by 1 to a minimum of 0
        mhi = np.maximum(mhi - 1/mhi_duration, 0)
        # Where there is motion, set the MHI to the maximum (current timestamp)
        mhi[motion_mask == 1] = timestamp

        prev_frame = current_frame

    # Normalize the MHI to the range of 0-255 for visualization
    mhi = np.uint8(np.clip((mhi - (timestamp - mhi_duration)) / mhi_duration, 0, 1) * 255)
    cap.release()
    return mhi

In [38]:
import cv2
import numpy as np
import os

def compute_rgbmhi(video_path, duration=1.0, threshold=64):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Failed to open video")
        return None
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Calculate the max value for MHI based on duration and FPS
    mhi_duration = int(fps * duration)  # Duration in frames

    # Initialize empty MHIs for each channel
    red_mhi = None
    green_mhi = None
    blue_mhi = None

    # Loop through the video
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video reached.")
            break

        # Convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate frame differences for each channel (assuming BGR order)
        next_frame_exists, next_frame = cap.read()
        if not next_frame_exists:
            print("End of video reached.")
            break
        bgr_diff = cv2.absdiff(frame, cv2.bitwise_not(next_frame))

        # Update MHIs for each channel
        red_mhi = update_mhi(red_mhi, bgr_diff[:,:,2], mhi_duration, threshold)
        green_mhi = update_mhi(green_mhi, bgr_diff[:,:,1], mhi_duration, threshold)
        blue_mhi = update_mhi(blue_mhi, bgr_diff[:,:,0], mhi_duration, threshold)
        

        
        frame_num += 1

    # Combine MHIs into RGB image (assuming brighter = more recent motion)
    rgbmhi = np.zeros_like(red_mhi, dtype=np.uint8)  # Create 3-channel image for RGB

    # Extract the first channel (assuming BGR order) for each assignment
    rgbmhi[blue_mhi > 0] = cv2.applyColorMap(blue_mhi, cv2.COLORMAP_BONE)[:, :, 0][blue_mhi > 0]
    rgbmhi[green_mhi > 0] = cv2.applyColorMap(green_mhi, cv2.COLORMAP_BONE)[:, :, 1][green_mhi > 0]  # Assuming COLORMAP_BONE for green tone
    rgbmhi[red_mhi > 0] = cv2.applyColorMap(red_mhi, cv2.COLORMAP_HOT)[:, :, 2][red_mhi > 0]

    cap.release()
    return rgbmhi

# Helper function to update a single channel MHI
def update_mhi(mhi, frame_diff, mhi_duration, threshold):
    if mhi is None:
        mhi = np.zeros_like(frame_diff, dtype=np.float32)

    mhi = np.maximum(mhi - 1/mhi_duration, 0)
    _, thresholded_diff = cv2.threshold(frame_diff, threshold, 1, cv2.THRESH_BINARY)
    mhi[thresholded_diff == 1] = mhi_duration

    # Normalize (you can adjust the normalization for color preference)
    mhi = np.uint8(np.clip(mhi / mhi_duration, 0, 1) * 255)
    return mhi


In [39]:

# Path to the folder containing videos
video_folder = r'/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor'

# List all files in the folder
video_files = os.listdir(video_folder)

# Iterate over video files
for video_file in video_files:
    # Check if the file is an MKV video
    if video_file.endswith('.mkv'):
        video_path = os.path.join(video_folder, video_file)
        # Compute the MHI for the provided video
        mhi_image = compute_rgbmhi(video_path)

        # Check if MHI was successfully computed
        if mhi_image is not None:
            # Create a folder named 'mhi' if it doesn't exist
            mhi_folder = os.path.join(video_folder, 'rgb-mhi-2')
            if not os.path.exists(mhi_folder):
                os.makedirs(mhi_folder)

            mhi_filename = f'mhi_{os.path.splitext(video_file)[0]}.png'
            mhi_path = os.path.join(mhi_folder, mhi_filename)
            cv2.imwrite(mhi_path, mhi_image)
            print(f"MHI saved: {mhi_path}")
        else:
            print("Failed to compute MHI for", video_file)


End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi-2/mhi_head_sensor_live_cam_135.png
End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi-2/mhi_head_sensor_live_cam_994.png
End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi-2/mhi_head_sensor_live_cam_841.png
End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi-2/mhi_head_sensor_live_cam_726.png
End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi-2/mhi_head_sensor_live_cam_1156.png
End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya

End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi-2/mhi_head_sensor_live_cam_59.png
End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi-2/mhi_head_sensor_live_cam_1511.png
End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi-2/mhi_head_sensor_live_cam_1163.png
End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi-2/mhi_head_sensor_live_cam_1231.png
End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi-2/mhi_head_sensor_live_cam_180.png
End of video reached.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antaly

### Sabitler renkli hareketliler beyaz

In [35]:
import cv2
import numpy as np
import os

def compute_rgbmhi(video_path, duration=1.0, threshold=64):
    # Vide dosyasını aç
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Hata: Video açılamadı")
        return None
    
    # FPS'i al
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Süre ve FPS'e dayalı olarak MHI için maksimum değeri hesapla
    mhi_duration = int(fps * duration)  # Karelerde süre

    # Her kanal için boş MHİ'leri başlat
    red_mhi = None
    green_mhi = None
    blue_mhi = None

    # Videoyu döngüye al
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Video sonuna ulaşıldı.")
            break

        # Gri tonlamaya dönüştür
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Her kanal için kare farklarını hesapla (BGR sırasıyla)
        next_frame_exists, next_frame = cap.read()
        if not next_frame_exists:
            print("Video sonuna ulaşıldı.")
            break
        bgr_diff = cv2.absdiff(frame, cv2.bitwise_not(next_frame))

        # Her kanal için MHİ'leri güncelle
        red_mhi = update_mhi(red_mhi, bgr_diff[:,:,2], mhi_duration, threshold)
        green_mhi = update_mhi(green_mhi, bgr_diff[:,:,1], mhi_duration, threshold)
        blue_mhi = update_mhi(blue_mhi, bgr_diff[:,:,0], mhi_duration, threshold)
        
        frame_num += 1

    # MHİ'leri birleştir ve RGB görüntüsü oluştur (Daha parlak = daha yeni hareket)
    rgbmhi = cv2.merge((blue_mhi, green_mhi, red_mhi))

    cap.release()
    return rgbmhi

# Tek kanallı bir MHİ'yi güncellemek için yardımcı işlev
def update_mhi(mhi, frame_diff, mhi_duration, threshold):
    if mhi is None:
        mhi = np.zeros_like(frame_diff, dtype=np.float32)

    mhi = np.maximum(mhi - 1/mhi_duration, 0)
    _, thresholded_diff = cv2.threshold(frame_diff, threshold, 1, cv2.THRESH_BINARY)
    mhi[thresholded_diff == 1] = mhi_duration

    # Normalize et (Renk tercihi için normalizasyonu ayarlayabilirsiniz)
    mhi = np.uint8(np.clip(mhi / mhi_duration, 0, 1) * 255)
    return mhi


# Videoların bulunduğu klasörün yolu
video_klasoru = r'/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor'

# Klasördeki tüm dosyaları listele
video_dosyalari = os.listdir(video_klasoru)

# Video dosyalarını işle
for video_dosyasi in video_dosyalari:
    # Dosyanın MKV bir video olup olmadığını kontrol et
    if video_dosyasi.endswith('.mkv'):
        video_yolu = os.path.join(video_klasoru, video_dosyasi)
        # Verilen video için MHİ'yi hesapla
        mhi_goruntusu = compute_rgbmhi(video_yolu)

        # MHİ başarıyla hesaplandı mı kontrol et
        if mhi_goruntusu is not None:
            # 'mhi' adında bir klasör oluştur (varsa)
            mhi_klasoru = os.path.join(video_klasoru, 'rgb-mhi')
            if not os.path.exists(mhi_klasoru):
                os.makedirs(mhi_klasoru)

            mhi_dosyasi_adi = f'mhi_{os.path.splitext(video_dosyasi)[0]}.png'
            mhi_yolu = os.path.join(mhi_klasoru, mhi_dosyasi_adi)
            cv2.imwrite(mhi_yolu, mhi_goruntusu)
            print(f"MHİ kaydedildi: {mhi_yolu}")
        else:
            print("MHİ hesaplanırken hata oluştu:", video_dosyasi)


Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi/mhi_head_sensor_live_cam_135.png
Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi/mhi_head_sensor_live_cam_994.png
Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi/mhi_head_sensor_live_cam_841.png
Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi/mhi_head_sensor_live_cam_726.png
Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi/mhi_head_sensor_live_cam_1156.png
Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4

Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi/mhi_head_sensor_live_cam_772.png
Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi/mhi_head_sensor_live_cam_59.png
Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi/mhi_head_sensor_live_cam_1511.png
Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi/mhi_head_sensor_live_cam_1163.png
Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_head_sensor/rgb-mhi/mhi_head_sensor_live_cam_1231.png
Video sonuna ulaşıldı.
MHİ kaydedildi: /home/muhlabws